In [ ]:
!pip install 'transformers[torch]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 28.1 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.3 MB/s eta 0:00:00


In [ ]:
from pprint import pprint

## **Preprocess a dataset.**

In [ ]:
# 1. Load the dataset from the hub.

from datasets import load_dataset
sms = load_dataset('sms_spam')
sms

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 5574
    })
})

In [ ]:
sms = sms['train'].train_test_split(train_size=0.85, seed=42, stratify_by_column='label')
sms

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 4737
    })
    test: Dataset({
        features: ['sms', 'label'],
        num_rows: 837
    })
})

In [ ]:
pprint(sms['train'][0])

{'label': 0,
 'sms': "She said,'' do u mind if I go into the bedroom for a minute ? '' "
        "''OK'', I sed in a sexy mood. She came out 5 minuts latr wid a "
        'cake...n My Wife,\n'}


In [ ]:
# 2. Tokenization using a tokenization_function and map method
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = 'bert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenization_function(inp):
    return tokenizer(inp['sms'], truncation=True)

tokenized_dataset = sms.map(tokenization_function, batched=True)
tokenized_dataset

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Map:   0%|          | 0/4737 [00:00<?, ? examples/s]

Map:   0%|          | 0/837 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sms', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4737
    })
    test: Dataset({
        features: ['sms', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 837
    })
})

In [ ]:
# Remove some of the columns that the model doesn't require.
tokenized_dataset = tokenized_dataset.remove_columns(['sms'])
tokenized_dataset = tokenized_dataset.rename_column('label', 'labels')
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4737
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 837
    })
})

In [ ]:
# 3. Dynamic padding using DataCollator.

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## **Training**

In [ ]:
# 1. Define TrainingArguments
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir='test_trainer', evaluation_strategy='epoch')

# 2. Define your Model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# 3. Define the compute metric
import numpy as np
!pip install evaluate
import evaluate

def compute_metrics(eval_preds):
    metrics = evaluate.combine([
        evaluate.load('accuracy'),
        evaluate.load('precision'),
        evaluate.load('recall'),
        evaluate.load('f1')
    ])
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metrics.compute(predictions=predictions, references=labels)

# 4. Define Trainer object
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
# 5. Train the transformer
trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.081200,0.068059,0.990442,0.990566,0.937500,0.963303
2,0.026900,0.079880,0.984468,0.938053,0.946429,0.942222
3,0.013100,0.068701,0.989247,0.972477,0.946429,0.959276


TrainOutput(global_step=1779, training_loss=0.03492282614941138, metrics={'train_runtime': 225.0315, 'train_samples_per_second': 63.151, 'train_steps_per_second': 7.906, 'total_flos': 396980624216340.0, 'train_loss': 0.03492282614941138, 'epoch': 3.0})

In [ ]:
classifier = AutoModelForSequenceClassification.from_pretrained('test_trainer/checkpoint-1500')
classifier

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("sentiment-analysis", model='test_trainer/checkpoint-1500')

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
sample = "Become a sub-broker today! Monthly payouts on a high revenue-sharing model with zero office expenses! txtby.me/tc/2GSjt9Fo Um9-ADMPL."

In [ ]:
sample = 'Dear MyMochi Member, The most awaited #MochiEOSS is here! Get Flat 50% off on Mochi & upto 50% off on other brands from 14 - 16 July. Sale preview for you on 13th July.'

In [ ]:
sample = "Hey cleon, How's your masters going in Canada?"

In [ ]:
classifier(sample)

[{'label': 'LABEL_1', 'score': 0.9992200136184692}]

In [ ]:
model.config.id2label

{0: 'LABEL_0', 1: 'LABEL_1'}